In [1]:
import numpy as np, pandas as pd

#get the submission file from GitHub
url='https://raw.githubusercontent.com/TRACK2-DATA-SCIENCE-2021/Track2_Challenges_Data/main/Challenge3/submissions/Challenge3_Day1Submission_sample.csv'
submission=pd.read_csv(url)
submission.tail()

,ID,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
15,16,91,-13,0,0,0,0,0,0,0,0,78
16,17,95,-7,0,0,0,0,0,0,0,0,88
17,18,99,-1,0,0,0,0,0,0,0,0,98
18,19,88,-25,0,0,0,0,0,0,0,0,63
19,20,20,-80,0,0,0,0,0,0,0,0,-60


In [2]:
import numpy as np

#Vector for quick addition of treatment
treatment_vec=np.array([0,1,1,5,30, 30,25,15,15,20,10])

#Sort sailors based on their predicted health at the end of day 2 with no treatment
day2health=submission['Health']+2*submission['Rate of health decline']
sickest_sailors=day2health.values.argsort(kind='mergesort')

#Put the thirty sickest (or all) sailors in beds
if submission.shape[0]<30:
  submission['Bed']=1
else:
  submission.loc[sickest_sailors[:30], 'Bed']=1

#Give the ten sickest (or all) sailors  ventilators 
if submission.shape[0]<10:
  submission['Ventilator']=1
else:
  submission.loc[sickest_sailors[:10], 'Ventilator']=1
  #Give the next ten sickest (or all) sailors sumplemental oxygen
  if submission.shape[0]<20:
    submission.loc[10:submission.shape[0], 'Supplemental oxygen']=1
  else:
    submission.loc[sickest_sailors[10:20], 'Supplemental oxygen']=1

#If health is greater than 90, the bonus points are worth more than health
submission.loc[lambda df:df['Health']+df['Rate of health decline']>90,
               ['Ventilator', 'Supplemental oxygen']]=0

#Recalculate total
perfect_fn=lambda x:100 if x>100 else x
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)
submission

,ID,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
0,1,100,-1,1,0,0,0,0,0,0,0,100
1,2,75,-1,1,0,0,0,0,0,1,0,99
2,3,65,-7,1,0,0,0,0,0,1,0,83
3,4,55,-25,1,0,0,0,0,0,1,0,55
4,5,33,-30,1,1,0,0,0,0,0,0,38
5,6,89,-50,1,0,0,0,0,0,1,0,64
6,7,90,-60,1,1,0,0,0,0,0,0,65
7,8,45,-65,1,1,0,0,0,0,0,0,15
8,9,100,-79,1,1,0,0,0,0,0,0,56
9,10,95,-88,1,1,0,0,0,0,0,0,42


In [3]:
#Set a goal health for each sailor that they will survive until Day 3
#If health decline is -50 or less, assume they'll need a ventilator or supplemental ox
goal_fn=lambda x:1-x if x>-50 else -34-x
goal_vec=submission['Rate of health decline'].apply(goal_fn)

In [4]:
#Apply Remdesivir to sailors who need it to reach goal health and recalculate health
need_Rem=np.where(submission['Total'].values<goal_vec-26)[-1]
submission.loc[need_Rem,'Remdesivir']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#Likewise for Dexamethasone
need_Dex=np.where(submission['Total'].values<goal_vec-21)[-1]
submission.loc[need_Dex,'Dexamethasone']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#Use Plasma and Chloroquine for those just healthy enough to not need dexamethasone
need_Plasma_Chloro=np.where(submission['Total'].values<goal_vec-16)[-1]
submission.loc[need_Plasma_Chloro,['Plasma', 'Chloroquine']]=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#Next option is need_Casirivimab
need_Casirivimab=np.where(submission['Total'].values<goal_vec-11)[-1]
submission.loc[need_Casirivimab,'Casirivimab']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#And lastly just plasma
need_Plasma=np.where(submission['Total'].values<goal_vec)[-1]
submission.loc[need_Plasma,'Plasma']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

submission

,ID,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
0,1,100,-1,1,0,0,0,0,0,0,0,100
1,2,75,-1,1,0,0,0,0,0,1,0,99
2,3,65,-7,1,0,0,0,0,0,1,0,83
3,4,55,-25,1,0,0,0,0,0,1,0,55
4,5,33,-30,1,1,0,0,0,0,0,0,38
5,6,89,-50,1,0,0,0,0,0,1,0,64
6,7,90,-60,1,1,0,0,0,0,0,0,65
7,8,45,-65,1,1,0,0,1,1,0,0,45
8,9,100,-79,1,1,0,0,0,0,0,0,56
9,10,95,-88,1,1,0,0,0,1,0,0,57


In [5]:
submission.to_csv('Challenge3_Day1Submission.csv')

In [6]:
#Check to make sure we haven't run out of anything
#It looks like we're all good
submission.sum(axis=0)

ID                         210
Health                    1394
Rate of health decline    -836
Bed                         20
Ventilator                  10
Remdesivir                   3
Dexamethasone                2
Plasma                       3
Casirivimab                  5
Supplemental oxygen          8
Chloroquine                  0
Total                     1355
dtype: int64